# Sensitivity analysis of population parameters

In [ ]:
[
    "copy_results_out_of_temp_dir=True\n",
    "# sub-directory of Results directory to copy into\n",
    "#sub_dir=\"Multiple_population_correlations\"  \n",
    "sub_dir='Systematic_correlation_results'\n",
    "#sub_dir='Test'\n",
    "interpret_as_max_values=False\n",
    "\n",
    "for length in[1000,2000,3000,4000,5000]:\n",
    "    for number_of_populations in [3,5,8,10]:\n",
    "        for size in [5,10,15,20]:\n",
    "            for event_prob in [0.0001,0.005,0.01,0.05,0.1,0.5]:\n",
    "                for noise_prob in [0.001,0.01,0.02,0.05,0.07,0.1,0.2]:\n",
    "                    for max_lag in [4,10,20,30,50,70]: # this is always interpreted as the maximum and a random selection made up to this value  \n",
    "                        for individuals_within_a_population_have_same_incidence in [False,True]:\n",
    "                            \n",
    "                            run_single_comparison(length=length,number_of_populations=number_of_populations,\n",
    "                     size=size,event_prob=event_prob,noise_prob=noise_prob,interpret_as_max_values=interpret_as_max_values,\n",
    "                     max_lag=max_lag,individuals_within_a_population_have_same_incidence=individuals_within_a_population_have_same_incidence,\n",
    "                      copy_results_out_of_temp_dir=copy_results_out_of_temp_dir,sub_dir=sub_dir,reclustering=None)"
  ]

In [3]:

copy_results_out_of_temp_dir=True


# sub-directory of Results directory to copy into

#sub_dir="Multiple_population_correlations"  
#sub_dir='Systematic_correlation_results'
sub_dir='Test'



length = 1234
number_of_populations=3
size=20
event_prob=0.05
noise_prob=0.05
interpret_as_max_values=False
max_lag=60   # this is always interpreted as the maximum and a random selection made up to this value

for individuals_within_a_population_have_same_incidence in [True]:
    run_single_comparison(length=length,number_of_populations=number_of_populations,
                     size=size,event_prob=event_prob,noise_prob=noise_prob,interpret_as_max_values=interpret_as_max_values,
                     max_lag=max_lag,individuals_within_a_population_have_same_incidence=individuals_within_a_population_have_same_incidence,
                      copy_results_out_of_temp_dir=copy_results_out_of_temp_dir,sub_dir=sub_dir,reclustering=None)

Initialising a random population None, size 1, with betas [20.0]
Betas for population None are [20.0]
Initialising a random population None, size 1, with betas [20.0]
Betas for population None are [20.0]
Initialising a random population None, size 1, with betas [20.0]
Betas for population None are [20.0]
Initialising for key Jill_noise
Expected 20 beta values, but only 1 values passed.  Initialising all time series with beta parameter 20.0
Initialising a random population Jill_noise, size 20, with betas [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]
Betas for population Jill_noise are [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]
Elapsed time: 0.011970996856689453
Initialising for key Jill
Creating lagging time series population based on single event time series
The poisson rates for lagging population Jill are [41 30 57 45 41 12 52 52 12 

,T,Z score std dev (v1 sigma),Z score mean (v1 sigma),Z score std dev (v2 sigma),Z score mean (v2 sigma)
0,1234,1.056,1.015,0.707,0.253


In [2]:
import testing_and_analysis_functions as analysis_func
import poisson_processes as pp
import pointwise_correlation as pc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Louvain
plt.rcParams['figure.figsize']=[20,10]

def run_single_comparison(length,number_of_populations,
                          size,event_prob,noise_prob,interpret_as_max_values,
                         max_lag,individuals_within_a_population_have_same_incidence,
                        copy_results_out_of_temp_dir,sub_dir='Multiple_population_correlations',
                         reclustering=None,verbose=False):
    
    if interpret_as_max_values:
        metaparams=analysis_func.create_random_metaparams(length=length,number_of_populations=number_of_populations,
                                                      max_size=size,max_lag=max_lag,max_event_prob=event_prob,max_noise_prob=noise_prob)
    else:
        metaparams=analysis_func.create_fixed_metaparams(length=length,number_of_populations=number_of_populations,
                                                     size=size,lag=max_lag,event_prob=event_prob,noise_prob=noise_prob)

    population_params=analysis_func.directly_initialise_multiple_populations(length=length,metaparams=metaparams,
                                                       use_fixed_means=individuals_within_a_population_have_same_incidence)

    #Copy metaparameters to csv file in TEMP directory
    metaparams['interpret size,noise and event probs as maximums']={' ':interpret_as_max_values}
    metaparams['same incidence for all members of a population']={' ':individuals_within_a_population_have_same_incidence}
    pd.DataFrame.from_dict(metaparams,orient='index').to_csv("{0}\meta_params.csv".format(pc.TEMP_DIR))
    # remove metaparams that are not population keys
    metaparams.pop('interpret size,noise and event probs as maximums')
    metaparams.pop('same incidence for all members of a population')
    
    # Copy individual population parameters to csv file in TEMP directory
    df=pd.DataFrame.from_dict(population_params,orient='index')
    df['prior process beta'] = df['prior poisson process'].apply(lambda x: None if type(x)==float else x.betas)
    df['mean lag list']=df['lag parameters'].apply(lambda x: None if type(x)==float else x['mus'])
    df=df.drop(['prior poisson process','lag parameters'],axis=1)
    df.to_csv("{0}\population_parameters.csv".format(pc.TEMP_DIR))

    
    
    #Intialise time series objects within their populations    
    mpp = pp.mixed_poisson_populations(length,population_params,verbose=verbose)
    #mpp.display(stats=True)

    # mix populations and extract the full array of time_series_objects
    ts_obj1=mpp.randomly_mix_populations(list(metaparams.keys()))
    number=len(ts_obj1)   # total number of all time series in all populations        
    ts_obj2=[]

    # compare v1 and v2 sigma measures and store correlations
    td,df=analysis_func.compare_inferred_and_known_means(xs=[length],number=number,ts_matrices=[ts_obj1,ts_obj2],
                                                    reclustering=None)
    display(df)

    # copy results out of pc.TEMP_DIR directory into given directory
    if copy_results_out_of_temp_dir:
        newly_created_sub_directory=analysis_func.copy_from_temp(dest_root_dir="{0}/{1}".format(pc.RESULTS_DIR,sub_dir))
    
    # create index entry for most recent run
    #with open("INDEX_FILE.csv") as file:
        
        
        
    
        
    
    if reclustering=='greedy Louvain':
        v1_df,v2_df=analysis_func.load_results_to_dfs(pc.TEMP_DIR)
        scores_v1={'version 1':Louvain.make_partition_and_score(v1_df,test_random_graph=False,pass_weights=True,verbose=False)}
        display(pd.DataFrame.from_dict(scores_v1,orient='index'))
        scores_v2={'version 2':Louvain.make_partition_and_score(v2_df,test_random_graph=False,pass_weights=True,verbose=False)}
        display(pd.DataFrame.from_dict(scores_v2,orient='index'))
    


In [1]:
import pointwise_correlation as pc
import pandas as pd
import Louvain
import testing_and_analysis_functions as analysis_func
v1_df,v2_df=analysis_func.load_results_to_dfs(pc.TEMP_DIR)
#v1_df=pd.read_csv("{0}/sigma_v1_correlations.csv".format(pc.TEMP_DIR))
print(v1_df.head(10))
#scores_v1=Louvain.make_partition_and_score(v1_df,test_random_graph=False,pass_weights=True,verbose=False)
#display(pd.DataFrame.from_dict({'v1':scores_v1},orient='index'))
#scores_v2=Louvain.make_partition_and_score(v2_df,test_random_graph=False,pass_weights=True,verbose=False)
#display(pd.DataFrame.from_dict({'v2':scores_v2},orient='index'))

df_results=v1_df
names=list(set(df_results['name1']).union(set(df_results['name2'])))
print(names)
#nodes=list(set(df_results['object1']).union(set(df_results['object2'])))
#print(df_results.head(40))
#print([df_results.loc[i,'object1'] for i in df_results.index[:10]])
#name_of_node={df_results.loc[i,'object1']:df_results.loc[i,'name1'] for i in df_results.index}
#id_test=df_results.loc[0,'object1']
#print(id_test)
#print(type(id_test))

name_of_node={}
for i in df_results.index:
    name_of_node[df_results.loc[i,'object2']]=df_results.loc[i,'name2']
    
print(name_of_node)

   p-value  Z-score name1 name2        object1        object2
0    0.098   -1.292  Owen   Tom  1556018621856  1555934688312
1    0.754    0.687  Owen  Rosa  1556018621856  1555934775840
2    0.689    0.493  Owen  Rosa  1556018621856  1555934774160
3    0.862    1.087  Owen  Rosa  1556018621856  1555934774832
4    0.923    1.427  Owen  Rosa  1556018621856  1555934773936
5    0.960    1.755  Owen  Rosa  1556018621856  1555934776624
6    0.551    0.128  Owen   Tom  1556018621856  1555934689992
7    0.946    1.611  Owen  Rosa  1556018621856  1555934773824
8    0.837    0.980  Owen   Tom  1556018621856  1555934690216
9    0.766    0.726  Owen  Rosa  1556018621856  1555934776176
['Rosa', 'Owen', 'Tom']
{1555934688312: 'Tom', 1555934775840: 'Rosa', 1555934774160: 'Rosa', 1555934774832: 'Rosa', 1555934773936: 'Rosa', 1555934776624: 'Rosa', 1555934689992: 'Tom', 1555934773824: 'Rosa', 1555934690216: 'Tom', 1555934776176: 'Rosa', 1555934689936: 'Tom', 1556018621240: 'Owen', 1555934690048: 'Tom',

In [3]:
import testing_and_analysis_functions as analysis_func
import pointwise_correlation as pc
analysis_func.copy_from_temp(dest_root_dir="{0}/Multiple_population_correlations".format(pc.RESULTS_DIR))

In [16]:
import pointwise_correlation as pc
df=pd.read_csv("{0}/meta_params.csv".format(pc.TEMP_DIR),index_col=0)
print(df.loc[df.index[0],'T'])


5000
